## Описание проекта

Сервис по продаже автомобилей с пробегом разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

# 1. Подготовка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
from sklearn.preprocessing import StandardScaler

На данном этапе я добавляю нужные для проекта библиотеки

In [2]:
data = pd.read_csv('/datasets/autos.csv')
print(data.shape)
#data.head()

(354369, 16)


Здесь я делаю вычитку и вывожу данные

In [3]:
data = data.query('RegistrationYear >= 1950')
data = data.query('RegistrationYear <= 2019')
data = data.query('Power > 50')
data = data.query('Power < 330')
data = data.query('Price > 1000')

data.shape
#data.isna().sum()

(239948, 16)

In [4]:

data = data.drop(['DateCrawled', 'DateCreated', 'LastSeen'], axis=1)

data = data.dropna()

print(data.shape)
print(data.info())
data.isna().sum()

(192317, 13)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 192317 entries, 3 to 354367
Data columns (total 13 columns):
Price                192317 non-null int64
VehicleType          192317 non-null object
RegistrationYear     192317 non-null int64
Gearbox              192317 non-null object
Power                192317 non-null int64
Model                192317 non-null object
Kilometer            192317 non-null int64
RegistrationMonth    192317 non-null int64
FuelType             192317 non-null object
Brand                192317 non-null object
NotRepaired          192317 non-null object
NumberOfPictures     192317 non-null int64
PostalCode           192317 non-null int64
dtypes: int64(7), object(6)
memory usage: 20.5+ MB
None


Price                0
VehicleType          0
RegistrationYear     0
Gearbox              0
Power                0
Model                0
Kilometer            0
RegistrationMonth    0
FuelType             0
Brand                0
NotRepaired          0
NumberOfPictures     0
PostalCode           0
dtype: int64

На данном этапе я удаляю не нужные столбцы, а также удаляю пропущенные значения

In [5]:
data_ohe = pd.get_dummies(data, drop_first=True)

Здесь я применяю метод прямого кодирования

In [6]:
df, df_test = train_test_split(data_ohe, test_size = 0.2, random_state=12345)

numeric = ['RegistrationYear', 'Power', 'Kilometer', 'RegistrationMonth', 'NumberOfPictures', 'PostalCode']


features_test = df_test.drop(['Price'], axis=1)
target_test = df_test['Price']

print(features_test.shape)
df_train, df_valid = train_test_split(df, test_size = 0.2, random_state=12345)

features_train = df_train.drop(['Price'], axis=1)
target_train = df_train['Price']

features_valid = df_valid.drop(['Price'], axis=1)
target_valid = df_valid['Price']

print(features_train.shape)
print(features_valid.shape)
print(target_train.shape)
print(target_valid.shape)
#features_train.head()

(38464, 304)
(123082, 304)
(30771, 304)
(123082,)
(30771,)


На данном этапе я разделяю данные на тренировочную, валидационную и тестовую выборки

In [7]:
scaler = StandardScaler()
scaler.fit(features_train[numeric])

features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])

1. Посмотрел признак RegistrationYear и отсек лишние и неправильные года
2. Также по другим признакам отсек лишние и несуществующие данные
3. Из таблицы удалил 3 признака DateCrawled, DateCreated, LastSeen
4. Пропуски в таблице удалил, при этом объем выборки остался достаточно большим для обучения модели

# 2. Обучение моделей

In [8]:
%%time
model = lgb.LGBMRegressor()
model.fit(features_train, target_train)

CPU times: user 7.5 s, sys: 240 ms, total: 7.74 s
Wall time: 7.76 s


LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

Здесь я провожу обучение модели и вывожу время обучения модели

In [9]:
%%time
predict_valid = model.predict(features_valid)
rmse = mean_squared_error(target_valid, predict_valid) ** 0.5
rmse

CPU times: user 533 ms, sys: 38 ms, total: 571 ms
Wall time: 509 ms


1678.0893143789312

Теперь я замеряю метрику  rmse и вывожу время, за которое происходит предсказывание модели и вычисление метрики

In [10]:
%%time
model_1 = lgb.LGBMRegressor(max_depth = 20, learning_rate = 0.05, feature_fraction = 0.8, n_estimators=300)
model_1.fit(features_train, target_train)

CPU times: user 18.9 s, sys: 267 ms, total: 19.1 s
Wall time: 19.4 s


LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              feature_fraction=0.8, importance_type='split', learning_rate=0.05,
              max_depth=20, min_child_samples=20, min_child_weight=0.001,
              min_split_gain=0.0, n_estimators=300, n_jobs=-1, num_leaves=31,
              objective=None, random_state=None, reg_alpha=0.0, reg_lambda=0.0,
              silent=True, subsample=1.0, subsample_for_bin=200000,
              subsample_freq=0)

Здесь я изменяю пару параметров обучения модели и вывожу время обучения

In [11]:
%%time
predict_valid_1 = model_1.predict(features_valid)
rmse_1 = mean_squared_error(target_valid, predict_valid_1) ** 0.5
rmse_1

CPU times: user 1.39 s, sys: 22.8 ms, total: 1.42 s
Wall time: 1.41 s


1641.5426846100222

Здесь я замеряю метрику rmse при новых параметрах и вывожу время, за которое происходит предсказывание модели и вычисление метрики

In [12]:
%%time
predict_test_1 = model_1.predict(features_test)
rmse_test = mean_squared_error(target_test, predict_test_1) ** 0.5
rmse_test

CPU times: user 1.7 s, sys: 55.2 ms, total: 1.75 s
Wall time: 1.8 s


1634.7782657461337

Здесь я замеряю метрику на тестовой выборке и вывожу время, за которое происходит предсказывание модели и вычисление метрики

In [13]:
pred_median = np.ones(features_valid.shape[0]) * target_train.median()
rmse_const = mean_squared_error(target_valid, pred_median) ** 0.5
rmse_const

4866.551591303901

Здесь я замеряю метрику на константной моделе и вывожу метрику rmse

# 3. Анализ моделей

Таким образом, значение метрики rmse при вычислении на тестовой выборке получилось 1635, скорость данного предсказания = 1.85 s, а время обучения модели = 17.4 s.